In [14]:
import pandas as pd
import numpy as np
from utils import utils
import functools as ft

In [15]:
df_papers = [pd.read_excel('papers_processed/papers_final.xlsx')]
df_papers.append(pd.read_excel('papers_processed/gabriel.xlsx'))
df_papers.append(pd.read_excel('papers_processed/karmanya_main_results_new.xlsx'))
df_papers.append(pd.read_excel('papers_processed/maik.xlsx'))
# df_papers.append(pd.read_excel('papers_processed/Paper2_data.xlsx'))
# df_papers.append(pd.read_excel('papers_processed/Paper9_data.xlsx'))
df_papers.append(pd.read_excel('papers_processed/Paper16_data.xlsx'))
df_papers.append(pd.read_excel('papers_processed/Paper18_data.xlsx'))
df_papers.append(pd.read_excel('papers_processed/Paper21_data.xlsx'))
df_papers.append(pd.read_excel('papers_processed/Paper29_data.xlsx'))
df_papers.append(pd.read_excel('papers_processed/Paper30_data.xlsx'))
df_papers.append(pd.read_excel('papers_processed/Paper33_data.xlsx'))
df_data = pd.read_csv('combined data/soil_grids_puls_agricast.csv')

df_papers

[                            doi        selection_type selection  \
 0      10.1088/1748-9326/aa67bd              latitude       <35   
 1   10.1016/j.agwat.2022.107645               soil ph       <7    
 2   10.1016/j.agwat.2022.107645               soil ph       >7    
 3   10.1016/j.agwat.2022.107645  yearly precipitation     <200    
 4   10.1016/j.agwat.2022.107645  yearly precipitation   200-400   
 5   10.1016/j.agwat.2022.107645  yearly precipitation   400-800   
 6   10.1016/j.agwat.2022.107645  yearly precipitation      >800   
 7   10.1016/j.agwat.2022.107645    yearly temperature       >12   
 8   10.1016/j.agwat.2022.107645    yearly temperature       <12   
 9   10.1016/j.agwat.2021.106771    yearly temperature        <9   
 10  10.1016/j.agwat.2021.106986             soil type     loamy   
 11  10.1016/j.agwat.2021.106986             soil type      clay   
 12  10.1016/j.agwat.2018.07.038               soil ph       < 7   
 13  10.1016/j.agwat.2018.07.038               s

In [16]:
df_data.head()

,Unnamed: 0,lon,lat,yearly_rain_6,yearly_rain_7,yearly_rain_8,yearly_rain_9,yearly_rain_10,yearly_rain_11,yearly_rain_12,...,bdod_30-60cm_mean,bdod_60-100cm_mean,bdod_100-200cm_mean,cec_0-5cm_mean,cec_5-15cm_mean,cec_15-30cm_mean,cec_30-60cm_mean,cec_60-100cm_mean,cec_100-200cm_mean,MostProbable
0,0,31.938307,26.862311,0.2,0.0,2.2,0.1,17.200000,1.5,1.0,...,139.3832,141.0469,138.6665,158.8610,157.0027,161.1208,162.1165,161.5760,166.0848,16.0
1,1,32.255928,26.862311,0.2,0.0,2.5,0.1,17.900000,1.4,1.2,...,139.7462,141.3125,138.7700,166.0894,163.1325,167.0549,167.7050,169.4801,174.7787,16.0
2,2,32.573549,26.862311,0.2,0.0,2.7,0.0,19.100000,0.8,1.1,...,141.6442,142.2828,140.8639,158.5099,155.0821,158.1587,160.7834,162.9597,167.9029,16.0
3,3,32.891170,26.862311,0.2,0.0,3.0,0.0,21.600000,0.6,1.1,...,137.0789,137.9260,135.5110,151.6080,149.9889,151.6562,154.6867,155.0006,155.3109,16.0
4,4,33.208791,26.862311,0.1,0.0,3.9,0.0,24.800001,0.4,1.2,...,143.2676,143.1493,143.0609,166.4694,159.5216,162.2418,162.3376,167.1551,168.4853,16.0


In [17]:
df_data['phh2o_0-5cm_mean'], df_data['phh2o_5-15cm_mean'],df_data['phh2o_15-30cm_mean'],df_data['phh2o_30-60cm_mean'],df_data['phh2o_60-100cm_mean'],df_data['phh2o_100-200cm_mean'] = df_data['phh2o_0-5cm_mean']/10, df_data['phh2o_5-15cm_mean']/10,df_data['phh2o_15-30cm_mean']/10,df_data['phh2o_30-60cm_mean']/10,df_data['phh2o_60-100cm_mean']/10,df_data['phh2o_100-200cm_mean']/10

In [18]:
def process_numerical(env_data, row, selection_env, temp):
    print(row['selection'])
    selection = row['selection']
    if "-" in selection:
        bounds = selection.split('-')
        # print(bounds)
        satisfying_rows = env_data[(env_data[selection_env] > float(bounds[0]))]
        satisfying_rows = satisfying_rows[(satisfying_rows[selection_env] < float(bounds[1]))]
        satisfying_rows['results_detailed'], satisfying_rows['results_actual'], satisfying_rows['farming_practice'] = row['results_detailed'], row['results_actual'], row['farming_practice']
        temp.append(satisfying_rows)
        # print(satisfying_rows)
    elif ">" in selection:
        bound = selection[1:]
        satisfying_rows = env_data[(env_data[selection_env] > float(bound))]
        satisfying_rows['results_detailed'], satisfying_rows['results_actual'], satisfying_rows['farming_practice'] = row['results_detailed'], row['results_actual'], row['farming_practice']
        temp.append(satisfying_rows)
    elif "<" in selection:
        bound = selection[1:]
        satisfying_rows = env_data[(env_data[selection_env] < float(bound))]
        satisfying_rows['results_detailed'], satisfying_rows['results_actual'], satisfying_rows['farming_practice'] = row['results_detailed'], row['results_actual'], row['farming_practice']
        temp.append(satisfying_rows)
    return temp

In [19]:
soil_type_map = {
    'Acrisols': 0,
    'Albeluvisols': 1,
    'Alisols': 2,
    'Andosols': 3,
    'Arenosols': 4,
    'Calcisols': 5,
    'Cambisols': 6,
    'Chernozems': 7,
    'Cryosols': 8,
    'Durisols': 9,
    'Ferralsols': 10,
    'Fluvisols': 11,
    'Gleysols': 12,
    'Gypsisols': 13,
    'Histosols': 14,
    'Kastanozems': 15,
    'Leptosols': 16,
    'Lixisols': 17,
    'Luvisols': 18,
    'Nitisols': 19,
    'Phaeozems': 20,
    'Planosols': 21,
    'Plinthosols': 22,
    'Podzols': 23,
    'Regosols': 24,
    'Solonchaks': 25,
    'Solonetz': 26,
    'Stagnosols': 27,
    'Umbrisols': 28,
    'Vertisols': 29
}

In [20]:
def filter_by_soil_type(env_data, row, temp):
    print(row['selection'])
    soil_type = row['selection']
    
    # Convert g/kg to percentage
    sand_pct = env_data['sand_0-5cm_mean'] / 10
    silt_pct = env_data['silt_0-5cm_mean'] / 10
    clay_pct = env_data['clay_0-5cm_mean'] / 10
    ph = env_data['phh2o_0-5cm_mean']
    condition = "THIS IS A WALLUE"
    # Define conditions for each soil type
    if soil_type.lower() == 'sandy':
        condition = (sand_pct > 85) & (silt_pct + 1.5 * clay_pct < 15)
    
    elif soil_type.lower() == 'loamy':
        condition = (
            (sand_pct <= 52) & 
            (silt_pct >= 28) & (silt_pct <= 50) & 
            (clay_pct >= 7) & (clay_pct <= 27)
        )
    
    elif soil_type.lower() == 'clay':
        condition = (sand_pct <= 45) & (silt_pct <= 40) & (clay_pct >= 40)
    elif soil_type.lower() == 'sandy,acidic':
        condition = (sand_pct > 85) & (silt_pct + 1.5 * clay_pct < 15) & (ph <7)
    elif soil_type.lower() == 'lixisols':
        satisfying_rows = env_data[env_data['MostProbable'] == soil_type_map['Lixisols']]
        satisfying_rows['results_detailed'], satisfying_rows['results_actual'], satisfying_rows['farming_practice'] = row['results_detailed'], row['results_actual'], row['farming_practice']
        temp.append(satisfying_rows)
        return temp
    elif soil_type.lower() == 'arenosols':
        satisfying_rows = env_data[env_data['MostProbable'] == soil_type_map['Arenosols']]
        satisfying_rows['results_detailed'], satisfying_rows['results_actual'], satisfying_rows['farming_practice'] = row['results_detailed'], row['results_actual'], row['farming_practice']
        temp.append(satisfying_rows)
        return temp
    elif soil_type.lower() == 'acrisols':
        satisfying_rows = env_data[env_data['MostProbable'] == soil_type_map['Acrisols']]
        satisfying_rows['results_detailed'], satisfying_rows['results_actual'], satisfying_rows['farming_practice'] = row['results_detailed'], row['results_actual'], row['farming_practice']
        temp.append(satisfying_rows)
        return temp
    else:
        print(
            f"Invalid soil_type '{soil_type}'. "
            "Must be 'sandy', 'loamy', or 'clay'"
        )
        return temp
    satisfying_rows = env_data[condition]
    satisfying_rows['results_detailed'], satisfying_rows['results_actual'], satisfying_rows['farming_practice'] = row['results_detailed'], row['results_actual'], row['farming_practice']
    temp.append(satisfying_rows)
    return temp

In [21]:
process_dict = {
    'yearly precipitation': 'yearly_rain_mean',
    'latitude': 'lat',
    'soil ph': 'phh2o_0-5cm_mean',
    'yearly temperature': 'yearly_avg_mean_temp_mean'
}

In [ ]:
def combine(env_data, paper_data):
    temp = []
    for idx, row in paper_data.iterrows():
        sel_type = row['selection_type'].lower()
        if sel_type in process_dict.keys():
            temp = process_numerical(env_data, row, process_dict[sel_type], temp)
        elif sel_type == "soil_type":
            temp = filter_by_soil_type(env_data, row, temp)
        elif sel_type == "region":
            pass
    combined = pd.DataFrame(columns=['results_detailed','farming_practice','results_actual'])   
    if len(temp) > 0:
        combined = pd.concat(temp, ignore_index=True)   
    return combined
            
                

In [23]:
df_papers[4]['selection_type'] = df_papers[4]['selection type']
df_papers[4]

,doi,selection type,selection,selection_information_page,farming_practice,farming_practice_compared,results_detailed,results_actual,results_information_page,crop_type,carbon_footprint,carbon_footprint_information_page,notes,selection_type
0,10.1371/journal.pone.0231740,yearly temperature,<10,"1,5",Crop Residue Return,No Straw (Residue Removal),crop_yield,"""+7.6""","5,8",Various (Wheat/Corn/etc.),NaN,NaN,Residue return is most effective in colder cli...,yearly temperature
1,10.1371/journal.pone.0231740,yearly temperature,10-15,"1,5",Crop Residue Return,No Straw (Residue Removal),crop_yield,"""+4.9""","5,8",Various (Wheat/Corn/etc.),NaN,NaN,Positive effect decreases as temperature rises.,yearly temperature
2,10.1371/journal.pone.0231740,yearly precipitation,>800,"1,4",Crop Residue Return,No Straw (Residue Removal),crop_yield,"""+9.5""","1,8",Various (Wheat/Corn/etc.),NaN,NaN,Highest yield benefit found in wet regions.,yearly precipitation
3,10.1371/journal.pone.0231740,soil ph,<6.5,"1,4",Crop Residue Return,No Straw (Residue Removal),crop_yield,"""+11.2""","1,9",Various (Wheat/Corn/etc.),NaN,NaN,Strongest soil-based response found in acidic ...,soil ph


In [24]:
combined = []
for idx, data in enumerate(df_papers):
    print("---------------------------------------------")
    print(idx)
    combined.append(combine(df_data, data))
combined

---------------------------------------------
0
<35
<7 
>7 
<200 
200-400
400-800
>800
>12
<12
<9
< 7
>7
---------------------------------------------
1
<30
---------------------------------------------
2
<10
10–20
10–20
>20
---------------------------------------------
3
30-60
<13
>13
<400
400–800
<6.5
6.5–7.5
>7.5
---------------------------------------------
4
<10
10-15
>800
<6.5
---------------------------------------------
5
---------------------------------------------
6
---------------------------------------------
7
10-20
---------------------------------------------
8
<5.5
>7.0
---------------------------------------------
9
>7.5 
<6.5 


C:\Users\artur\AppData\Local\Temp\ipykernel_18056\1100590012.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  satisfying_rows['results_detailed'], satisfying_rows['results_actual'], satisfying_rows['farming_practice'] = row['results_detailed'], row['results_actual'], row['farming_practice']
C:\Users\artur\AppData\Local\Temp\ipykernel_18056\1100590012.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  satisfying_rows['results_detailed'], satisfying_rows['results_actual'], satisfying_rows['farming_pract

[       Unnamed: 0        lon        lat  yearly_rain_6  yearly_rain_7  \
 0               0  31.938307  26.862311        0.20000        0.00000   
 1               1  32.255928  26.862311        0.20000        0.00000   
 2               2  32.573549  26.862311        0.20000        0.00000   
 3               3  32.891170  26.862311        0.20000        0.00000   
 4               4  33.208791  26.862311        0.10000        0.00000   
 ...           ...        ...        ...            ...            ...   
 83528       10306  53.218914  52.907233      336.10016      402.40012   
 83529       10308  53.854156  52.907233      317.50020      394.20004   
 83530       10501  50.995567  53.224854      366.40027      402.50000   
 83531       10502  51.313188  53.224854      355.90024      402.20016   
 83532       10503  51.630809  53.224854      360.60016      409.90033   
 
        yearly_rain_8  yearly_rain_9  yearly_rain_10  yearly_rain_11  \
 0            2.20000        0.10000  

In [25]:
combined_fin = pd.concat(combined, ignore_index=True)  
combined_fin

,Unnamed: 0,lon,lat,yearly_rain_6,yearly_rain_7,yearly_rain_8,yearly_rain_9,yearly_rain_10,yearly_rain_11,yearly_rain_12,...,cec_0-5cm_mean,cec_5-15cm_mean,cec_15-30cm_mean,cec_30-60cm_mean,cec_60-100cm_mean,cec_100-200cm_mean,MostProbable,results_detailed,results_actual,farming_practice
0,0.0,31.938307,26.862311,0.20000,0.00000,2.20000,0.10000,17.200000,1.50000,1.00000,...,158.86100,157.00270,161.12080,162.11650,161.57600,166.08480,16.0,yield,"""+25""",biochar amendment
1,1.0,32.255928,26.862311,0.20000,0.00000,2.50000,0.10000,17.900000,1.40000,1.20000,...,166.08940,163.13250,167.05490,167.70500,169.48010,174.77870,16.0,yield,"""+25""",biochar amendment
2,2.0,32.573549,26.862311,0.20000,0.00000,2.70000,0.00000,19.100000,0.80000,1.10000,...,158.50990,155.08210,158.15870,160.78340,162.95970,167.90290,16.0,yield,"""+25""",biochar amendment
3,3.0,32.891170,26.862311,0.20000,0.00000,3.00000,0.00000,21.600000,0.60000,1.10000,...,151.60800,149.98890,151.65620,154.68670,155.00060,155.31090,16.0,yield,"""+25""",biochar amendment
4,4.0,33.208791,26.862311,0.10000,0.00000,3.90000,0.00000,24.800001,0.40000,1.20000,...,166.46940,159.52160,162.24180,162.33760,167.15510,168.48530,16.0,yield,"""+25""",biochar amendment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204104,18156.0,29.397339,70.694009,402.20010,406.59998,343.70016,389.80005,535.400150,445.50006,440.89990,...,311.10200,254.28140,244.01454,228.84657,220.20251,216.07089,6.0,crop_yield,"""+5""",Inhibitors (IS)
204105,18157.0,29.714960,70.694009,400.80008,406.99990,341.80002,388.20010,531.400100,448.19995,440.30000,...,303.13397,254.39972,234.21259,218.98473,205.91713,197.77560,6.0,crop_yield,"""+5""",Inhibitors (IS)
204106,18158.0,30.032581,70.694009,402.40024,416.40005,337.30008,387.59998,516.600160,449.40030,433.90000,...,306.09308,253.44981,232.27971,219.31424,214.39210,208.50879,6.0,crop_yield,"""+5""",Inhibitors (IS)
204107,18159.0,30.350202,70.694009,409.60016,421.70013,341.70013,392.90024,516.600200,450.50027,437.60013,...,306.60095,267.67460,226.02970,216.20903,205.11995,191.43942,6.0,crop_yield,"""+5""",Inhibitors (IS)


In [27]:
combined_fin.to_csv('example.csv', index=False)